In [1]:
import numpy as np
import pandas as pd
import gc
train=pd.read_csv('C:\\Users\\Pratik\\Desktop\\Assignments\\CS584\\Project\\Data\\train.csv',sep=',')
test=pd.read_csv('C:\\Users\\Pratik\\Desktop\\Assignments\\CS584\\Project\\Data\\test.csv',sep=',')

PHash Matching

In [2]:
sub=pd.DataFrame(test['posting_id'])
sub['phash']=[' '.join(list(test.loc[test.image_phash==test.iloc[i].image_phash].posting_id)) for i in range(len(test))]

TFIDF with KNN - 21 neighbors

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
#from nltk.corpus import stopwords
import string
words=(train['title'].append(test['title'])).str.lower().str.translate(str.maketrans('','',string.punctuation)).str.split().explode().value_counts()
words=list(set(words[words>1].index.values.tolist()))#-set(stopwords.words('english')))
tfidf=TfidfVectorizer(use_idf=True,vocabulary=words)
X_train=pd.DataFrame(tfidf.fit_transform(train['title']).toarray(),columns=tfidf.get_feature_names())
y_train=train['label_group']
X_test=pd.DataFrame(tfidf.fit_transform(test['title']).toarray(),columns=tfidf.get_feature_names())

In [4]:
from sklearn.neighbors import KNeighborsClassifier
knn=KNeighborsClassifier(n_neighbors=21)
knn.fit(X_train,y_train)
y_pred=knn.predict(X_test)

In [5]:
temp=pd.DataFrame(test['posting_id'])
temp['pred']=y_pred
sub['tfidf']=temp['pred'].map(temp.groupby('pred').posting_id.agg('unique')) #matching posting id against predicted labels

Memory Cleanup

In [7]:
del words,tfidf,TfidfVectorizer,string,X_train,y_train,X_test,knn,KNeighborsClassifier,y_pred,temp
gc.collect()

40

Train Test Split [80-20] #common code used with image and text as required

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X_train,y_train,test_size=0.2,shuffle=True)

In [ ]:
from sklearn.metrics import f1_score
print(f1_score(y_test,y_pred,average='weighted'))

Image Preprocessing and stored in Matrix (64 x 64 grayscale)

In [ ]:
def img_pre(img):
    return np.ndarray.flatten(cv2.GaussianBlur(cv2.resize(img,(64,64)),(15,15),0)) 

from os.path import join
import cv2
trvec=[]
lab=[]
for i in train[['image','label_group']].drop_duplicates().iterrows():
    trvec.append(img_pre(cv2.imread(join('C:\\Users\\Pratik\\Desktop\\Assignments\\CS584\\Project\\Data\\train_images',i[1][0]),0)))
    lab.append(i[1][1])
trvec=pd.DataFrame(trvec)
lab=pd.DataFrame(lab)

In [ ]:
trvec.to_csv(path_or_buf='C:\\Users\\Pratik\\Desktop\\Assignments\\CS584\\Project\\Gen\\training_image_vector.csv',header=None,index=False,index_label=None,mode='w')
lab.to_csv(path_or_buf='C:\\Users\\Pratik\\Desktop\\Assignments\\CS584\\Project\\Gen\\training_image_label.csv',header=None,index=False,index_label=None,mode='w')

In [5]:
gc.collect()

50

Load Image data

In [ ]:
trvec=pd.read_csv('C:\\Users\\Pratik\\Desktop\\Assignments\\CS584\\Project\\Gen\\training_image_vector.csv',sep=',',header=None)
lab=pd.read_csv('C:\\Users\\Pratik\\Desktop\\Assignments\\CS584\\Project\\Gen\\training_image_label.csv',sep=',',header=None)

In [ ]:
import cv2
from os.path import join
tsvec=pd.DataFrame([np.ndarray.flatten(cv2.GaussianBlur(cv2.resize(cv2.imread(join('C:\\Users\\Pratik\\Desktop\\Assignments\\CS584\\Project\\Data\\test_images',j),0),(64,64)),(15,15),0)) for j in test['image']])

Encode labels

In [ ]:
from sklearn.preprocessing import LabelEncoder as LE
y=pd.DataFrame(LE().fit_transform(lab))

Image Processing

In [ ]:
trvec/=255 #normalize data
tsvec/=255

from sklearn.decomposition import PCA #reduce dimensionality
pca=PCA(0.95)#n_components=50)
pca.fit(trvec)
train_img=pca.transform(trvec)
test_img=pca.transform(tsvec)

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
X_train=LDA(n_components=2).fit_transform(trvec,lab)

Image classification

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf=RandomForestClassifier(n_estimators=100)
y_pred_rf=rf.fit(train_img,lab).predict(test_img)

In [ ]:
from xgboost import XGBClassifier
model=XGBClassifier(use_label_encoder=True)
y_pred_xgb=model.fit(train_img,lab).predict(test_img)

In [ ]:
tr_des=[]
descr=[]
orb = cv2.ORB_create()
for i in train['image']:
    im=cv2.resize(cv2.imread(join('C:\\Users\\Pratik\\Desktop\\Assignments\\CS584\\Project\\Data\\train_images',i)),(128,128))
    key,des=orb.detectAndCompute(im,None)
    tr_des.append(des)
for j in test['image']:
    im=cv2.resize(cv2.imread(join('C:\\Users\\Pratik\\Desktop\\Assignments\\CS584\\Project\\Data\\test_images',j)),(128,128))
    key,des=orb.detectAndCompute(im,None)
    descr.append(des)

In [ ]:
del orb,im,key,des
gc.collect()

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers,models
cnn = models.Sequential([
    layers.Conv2D(filters=64,kernel_size=(3,3),activation='relu',input_shape=(64,64,1)),
    layers.MaxPooling2D((2,2)),
    layers.Flatten(),
    layers.Dense(64,activation='relu'),
    layers.Dense(len(np.unique(lab)),activation='softmax')])
cnn.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [ ]:
cnn.fit(np.reshape(np.asarray(trvec),(32460,64,64,1)),y,epochs=10)
y_pred=cnn.predict(np.asarray(tsvec))

Image Preprocessing and stored in Matrix (128 x 128 BGR)

In [ ]:
#128x128 rgb images trial
trvec=[]
lab=[]
for i in train[['image','label_group']].drop_duplicates().iterrows():
    trvec.append(cv2.resize(cv2.imread(join('C:\\Users\\Pratik\\Desktop\\Assignments\\CS584\\Project\\Data\\train_images',i[1][0])),(128,128)))
    lab.append(i[1][1])
#trvec=pd.DataFrame(trvec)
lab=pd.DataFrame(lab)
tsvec=[cv2.resize(cv2.imread(join('C:\\Users\\Pratik\\Desktop\\Assignments\\CS584\\Project\\Data\\test_images',j)),(128,128)) for j in test['image']]

In [ ]:
trvec.to_csv(path_or_buf='C:\\Users\\Pratik\\Desktop\\Assignments\\CS584\\Project\\Gen\\training_image_vector.csv',header=None,index=False,index_label=None,mode='w')
lab.to_csv(path_or_buf='C:\\Users\\Pratik\\Desktop\\Assignments\\CS584\\Project\\Gen\\training_image_label.csv',header=None,index=False,index_label=None,mode='w')

ANN

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers,models
ann=models.Sequential([layers.Flatten(input_shape=(128,128,3)),
        layers.Dense(3000, activation='relu'),
        layers.Dense(1000, activation='relu'),
        layers.Dense(11014, activation='sigmoid')])
ann.compile(optimizer='SGD',loss='sparse_categorical_crossentropy',metrics=['accuracy'])
ann.fit(np.asarray(trvec),y,epochs=10)
y_pred=ann.predict(np.asarray(tsvec))

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers,models
cnn = models.Sequential([
    layers.Conv2D(filters=128,kernel_size=(3,3),activation='relu',input_shape=(128,128,3)),
    layers.MaxPooling2D((2,2)),
    layers.Flatten(),
    layers.Dense(64,activation='relu'),
    layers.Dense(len(np.unique(lab)),activation='softmax')])

In [ ]:
cnn.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [ ]:
cnn.fit(np.asarray(trvec),y,epochs=5)
y_pred=cnn.predict(np.asarray(tsvec))

Sub file

In [ ]:
subm=pd.DataFrame(test['posting_id'])
subm['pred']=y_pred
subm['matches']=subm.groupby('pred')['posting_id'].head(50)
subm=subm.drop(columns='pred')
subm.to_csv(path_or_buf='C:\\Users\\Pratik\\Desktop\\submission.csv',index=False,index_label=None,mode='w')

Combine predictions

In [ ]:
#pending
sub['matches']=#(1.phash) (2. image) (3. tfidf)

In [112]:
sub=sub.drop(columns=['phash','tfidf'])
sub.to_csv(path_or_buf='C:\\Users\\Pratik\\Desktop\\submission.csv',index=False,index_label=None,mode='w')